In [7]:
public_market_api_url = "https://bittrex.com/api/v2.0/pub/market/"
public_markets_api_url = "https://bittrex.com/api/v2.0/pub/markets/"
from datetime import datetime
ts=str(int(datetime.now().strftime('%s'))*1000)
import json
from influxdb import InfluxDBClient
import requests

def fetch_data(market,interval):
    final_api_str='GetTicks'
    param_dict = {"marketName":market,"tickInterval":interval,'_':ts}
    r=requests.get(public_market_api_url+final_api_str,params=param_dict)
    #print r.content
    return json.loads(r.content)

def get_all_coins():
    final_api_str='GetMarketSummaries'
    param_dict = {'_':ts}
    r=requests.get(public_markets_api_url+final_api_str,params=param_dict)
    return r.content


def publish_values_to_influx(market_name=None,symbol=None,currency=None,interval=None):
    data=fetch_data(market_name,interval)
    client = InfluxDBClient('52.172.212.232', 8086, '', '', 'cryptotracker')
    total_json=[]
    for row in data['result']:
        default_json_body = {
            "measurement": "crypto",
            "tags": {
                "exchange": "bittrex",
                "symbol":symbol,
                "currency":currency,
                "sample":interval
            },
            "fields": {
                "low": None,
                "high":None,
                "open":None,
                "close":None,
                "volume":None,
                "bv":None
            },
            "time":None            
        }
        default_json_body['fields']["low"]=row["L"]
        default_json_body['fields']["high"]=row["H"]
        default_json_body['fields']["open"]=row["O"]
        default_json_body['fields']["close"]=row["C"]
        default_json_body['fields']["volume"]=row["V"]
        default_json_body['fields']["bv"]=row["BV"]
        default_json_body['time']=row["T"]+"Z"
        total_json.append(default_json_body)
    print ("to send data",total_json)
    # Returns True, if sent successfully.
    status=client.write_points(total_json)
    print (status)
['fiveMin','hour','day']

# """
# INSERT crypto,exchange="bitrex",symbol="xrp",currency="btc",sample="5min" high=0.00001616,
# low=0.00001609,open=0.00001612,close=0.00001612,volume=125318.77478722,bv=2.01998059
# """


['fiveMin', 'hour', 'day']

In [2]:
fetch_data('BTC-XRP','fiveMin')
data=json.loads(get_all_coins())

In [ ]:
print(len(data['result']))
print("--------")
item=data['result'][0]
item
publish_values_to_influx(market_name=item['Market']['MarketName'],
                         symbol=item['Market']['MarketCurrency'],
                         currency=item['Market']['BaseCurrency'],
                         interval='fiveMin')
# for item in data['result']:
#     print item['Market']['MarketCurrency'],item['Market']['BaseCurrency'],item['Market']['MarketName']

In [ ]:
from telethon import TelegramClient

# These example values won't work. You must get your own api_id and
# api_hash from https://my.telegram.org, under API Development.
api_id = 100791
api_hash = ''
phone = ''
client = TelegramClient('session_name', api_id, api_hash)
client.connect()

# If you already have a previous 'session_name.session' file, skip this.
client.sign_in(phone=phone)

In [ ]:
me = client.sign_in(code=42858)